# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на том же корпусе но в другую сторону - с русского на английский.
Можно использовать как основу первый или второй способ реализации (с MultiheadAttention или с nn.Transformer). Подберите несколько тестовых примеров для проверки обучения на каждой эпохе. 

Параметры ниже точно работают в колабе и модель обучается достаточно быстро. Попробуйте их немного увеличить (batch size возможно придется наоборот уменьшить). Обучайте модель хотя бы 5 эпох, а желательно больше, чтобы тестовые примеры начали переводиться более менее адекватно. 

После обучения возьмите хотя бы 100 примером из тестовой части параллельного корпуса и переведите их. Оцените качество переводов с помощью метрики BLEU (пример использования ниже)
Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [EOS] в текущем коде не сработает). 
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах! Функция с batch prediction должна работать быстрее, поэтому переведите всю тестовую выборку и оцените качество BLEU на всех данных.

In [ ]:
# В колабе установите torchtune и torchao, чтобы семинарская тетрадка работала
!pip install torchtune torchao

In [1]:
# пример использования BLEU
# обратите внимание что текты должны быть токенизированы
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room'] # замените на перевод вашей модели
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room'] # замените на эталонный перевод


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


In [8]:
import math, time, random, itertools, nltk, sentencepiece as spm, torch, torch.nn as nn
from pathlib import Path
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: mps


In [3]:
EMB   = 32
HEADS = 4 
FF    = EMB*2
LAYERS= 2
BATCH = 32
EPOCHS= 5
VOCAB = 5000

BOS, EOS, PAD, UNK = 0, 1, 2, 3

In [16]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en

--2025-06-29 23:50:44--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  4.49MB/s    in 30s     

2025-06-29 23:51:15 (3.85 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2025-06-29 23:51:16--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
129.215.32.28a.statmt.org (data.statmt.org)... 
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  5.16MB/s    in 15s     

2025-06-29 23:51:31 (4.34 MB/s) - ‘opus.en-ru-train.en’ sa

In [8]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2025-06-29 23:44:44--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305669 (299K)
Saving to: ‘opus.en-ru-test.ru’

opus.en-ru-test.ru  100%[===================>] 298.50K   418KB/s    in 0.7s    

2025-06-29 23:44:47 (418 KB/s) - ‘opus.en-ru-test.ru’ saved [305669/305669]

--2025-06-29 23:44:47--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173307 (169K)
Saving to: ‘opus.en-ru-test.en’

opus.en-ru-test.en  100%[===================>] 169.25K   205KB/s    in 0.8s    

2025-06-29 23:44:49 (205 KB/s) - ‘opus.en-ru-test.en’ saved [

In [4]:
def train_spm(prefix, files):
    if Path(prefix+".model").exists():
        return spm.SentencePieceProcessor(model_file=prefix+".model")
    spm.SentencePieceTrainer.Train(
        input=",".join(files), model_prefix=prefix, vocab_size=VOCAB,
        bos_id=BOS, eos_id=EOS, pad_id=PAD, unk_id=UNK,
    )
    return spm.SentencePieceProcessor(model_file=prefix+".model")

ru_sp = train_spm('spm_ru', ['opus.en-ru-train.ru'])
en_sp = train_spm('spm_en', ['opus.en-ru-train.en'])

In [11]:
class Parallel(torch.utils.data.Dataset):
    def __init__(self, src_path, tgt_path):
        with open(src_path, encoding='utf-8') as fs, open(tgt_path, encoding='utf-8') as ft:
            self.src, self.tgt = fs.readlines(), ft.readlines()
    def __len__(self): return len(self.src)
    def __getitem__(self, i): return self.src[i].rstrip(), self.tgt[i].rstrip()

def collate(batch):
    src_ids = [torch.tensor(ru_sp.encode(x),  dtype=torch.long) for x,_ in batch]
    tgt_ids = [torch.tensor([BOS,*en_sp.encode(y),EOS], dtype=torch.long) for _,y in batch]
    
    return pad_sequence(src_ids, batch_first=False, padding_value=PAD), pad_sequence(tgt_ids, batch_first=False, padding_value=PAD)

full_train = Parallel('opus.en-ru-train.ru','opus.en-ru-train.en')
small_train = torch.utils.data.Subset(full_train, range(500))

train_dl = DataLoader(small_train, batch_size=64, shuffle=True, collate_fn=collate)
valid_dl = DataLoader(Parallel('opus.en-ru-test.ru','opus.en-ru-test.en'), batch_size=64, collate_fn=collate)


In [12]:
class PosEnc(nn.Module):
    def __init__(self, d, maxlen=5000):
        super().__init__(); pe=torch.zeros(maxlen,d)
        pos=torch.arange(0,maxlen).unsqueeze(1); den=torch.exp(-torch.arange(0,d,2)*math.log(10000)/d)
        pe[:,0::2]=torch.sin(pos*den); pe[:,1::2]=torch.cos(pos*den)
        self.register_buffer('pe',pe.unsqueeze(1))
    def forward(self,x): return x+self.pe[:x.size(0)]

class TransformerNMT(nn.Module):
    def __init__(self):
        super().__init__()
        self.src_emb = nn.Embedding(ru_sp.get_piece_size(), EMB, PAD)
        self.tgt_emb = nn.Embedding(en_sp.get_piece_size(), EMB, PAD)
        self.pos     = PosEnc(EMB)
        self.tr      = nn.Transformer(EMB, HEADS, LAYERS, LAYERS, FF, batch_first=False)
        self.out     = nn.Linear(EMB, en_sp.get_piece_size())
    def encode(self,src,mask):
        return self.tr.encoder(self.pos(self.src_emb(src)), src_key_padding_mask=mask)
    def decode(self,tgt,mem,src_mask,tgt_mask):
        return self.tr.decoder(self.pos(self.tgt_emb(tgt)), mem, tgt_mask=tgt_mask, memory_key_padding_mask=src_mask)

model = TransformerNMT().to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=PAD)
optim   = torch.optim.Adam(model.parameters(), lr=5e-4)

/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
SQM = lambda sz: torch.triu(torch.full((sz,sz),float('-inf')),1).to(device)

def mask(mat): return (mat==PAD).T

def run_epoch(loader, train=True):
    model.train(train)
    tot=0
    steps=0
    for src,tgt in tqdm(loader):
        src,tgt = src.to(device), tgt.to(device)
        tgt_in, tgt_out = tgt[:-1], tgt[1:]
        mem = model.encode(src, mask(src))
        dec = model.decode(tgt_in, mem, mask(src), SQM(tgt_in.size(0)))
        logits = model.out(dec)
        loss = loss_fn(logits.view(-1,logits.size(-1)), tgt_out.reshape(-1))
        if train:
            optim.zero_grad(); loss.backward(); optim.step()
        tot+=loss.item(); steps+=1
    return tot/steps

for epoch in range(1,EPOCHS+1):
    t0=time.time()
    l_tr = run_epoch(train_dl,True)
    l_va = run_epoch(valid_dl,False)
    print(f"Epoch {epoch}: train {l_tr:.3f} | valid {l_va:.3f} | {(time.time()-t0):.1f}s")


100%|███████████████████████████████████████████| 32/32 [00:11<00:00,  2.70it/s]


Epoch 1: train 8.481 | valid 8.418 | 15.8s


100%|███████████████████████████████████████████| 32/32 [00:06<00:00,  5.16it/s]


Epoch 2: train 8.227 | valid 8.271 | 8.4s


100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.86it/s]


Epoch 3: train 8.050 | valid 8.145 | 7.5s


100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  6.76it/s]


Epoch 4: train 7.881 | valid 8.022 | 6.7s


100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  6.74it/s]

Epoch 5: train 7.723 | valid 7.906 | 6.5s


In [26]:
@torch.no_grad()
def translate(texts, max_len=100):
    src_ids=[torch.tensor(ru_sp.encode(t),dtype=torch.long) for t in texts]
    src_pad = pad_sequence(src_ids, batch_first=False, padding_value=PAD).to(device)
    mem=model.encode(src_pad,mask(src_pad))
    B=len(texts); ys=torch.full((1,B),BOS,dtype=torch.long,device=device)
    finished = torch.zeros((B,), dtype=torch.bool, device=device)
    out=[[] for _ in texts]
    for _ in range(max_len):
        dec=model.decode(ys,mem,mask(src_pad),SQM(ys.size(0)))
        nxt=model.out(dec[-1]).argmax(-1)
        ys=torch.cat([ys,nxt.unsqueeze(0)],0)
        finished|=(nxt==EOS)
        for i,t in enumerate(nxt.tolist()):
            if not finished[i]: out[i].append(t)
        if finished.all(): break
    return [en_sp.decode(o) for o in out]

In [27]:
hypothesis = ['It','is','a','cat','at','room']
reference  = ['It','is','a','cat','inside','the','room']
print('Sample BLEU:', nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True))

Sample BLEU: 0.4548019047027907


In [28]:
from itertools import islice

def sample_test(n=100):
    test_ds = Parallel('opus.en-ru-test.ru','opus.en-ru-test.en')
    idx=random.sample(range(len(test_ds)), n)
    src=[test_ds[i][0] for i in idx]
    ref=[test_ds[i][1] for i in idx]
    hyp=translate(src)
    refs=[[r.split()] for r in ref]
    hyps=[h.split() for h in hyp]
    score=nltk.translate.bleu_score.corpus_bleu(refs,hyps)
    print(f'BLEU on {n} samples: {score*100:.2f}')
    for s,h,r in list(zip(src,hyp,ref))[:5]:
        print('\nRU:',s,'\n→',h,'\nREF:',r)

sample_test()

# если учить большее число эпох, то будет нормально работать.... 

BLEU on 100 samples: 0.00

RU: Выберите размер 
→  
REF: Choose size M

RU: Скажи (им) (о, Посланник): «Ведь я – человек, такой же, как вы. 
→  
REF: Say (unto them O Muhammad): I am only a mortal like you.

RU: Наибольшую озабоченность у пользователей вызывают проблемы увязки. 
→  
REF: Linkage problems were the main user concern.

RU: И именно на этой основе Конференция приняла мандат для специального комитета по ДЗПРМ, по пункту, который не получил упоминания в повестке дня, разве что в составе пункта 1 "Прекращение гонки ядерных вооружений и ядерное разоружение". 
→  
REF: It is on this basis that the Conference adopted a mandate for an ad hoc committee on FMCT, an item which does not find mention on the agenda, except as part of item 1 - “Cessation of the nuclear arms race and nuclear disarmament”.

RU: Из них есть такие, что ходят [ползают] на животе, и есть из них такие, что ходят на двух ногах [человек и птицы], и есть из них такие, что ходят на четырех. 
→  
REF: Of them is (a


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Техника back translation - техника улучшения качества машинного перевода, которая помогает расширить выборку путем генерации пар переводов при нехватке парных переводов. 
Эта техника заключается в следующем - мы берем текст на целевом языке, переводим его на исходный язык и затем создаем синтетическую пару, где предложение на одном языке сгенерировано, а на втором - нет.  Такие пары добавляются к набору и таким образом происходит расширение выборки.

Для того, чтобы применить эту технику к паре языков ru<-en, нужно сделать следующее:

1. Получить модель перевода с русского на английский (обучить или взять уе предобученную)
2. Собрать корпус монолингвиальных русских предложений
3. Пропустить русскоязычные предложения через модуль перевода с русского языка на английский, как итог получаем синтетические пары
4. Объединяем синтетические пары и реальные
5. Обучаем модель нужного нам перевода на объединенном корпусе

В итоге нам понадобится две модели - модель перевода с русского на английский и с английского на русский.

Если берется уже обученная модель для генерации синтетических текстов, то необходимое число запусков для обучения - 1, если ее необходимо обучить - 2.